In [1]:
import sqlalchemy as sa
import urllib 
import pandas as pd 
import psycopg2
from sqlalchemy import text
from conector import enginok

***LECTURA DE ARCHIVO.CSV Y CREAMOS EL DATAFRAME PARA TRABAJAR SOBRE EL MISMO***

In [3]:
dataB = pd.read_csv('../bibliotecas/Aug-2022/22-08-2022 Bibliotecas.csv')
df_Biblioteca = pd.DataFrame(dataB)
df_Biblioteca.sample(2)

,Cod_Loc,IdProvincia,IdDepartamento,Observacion,CategorÃ­a,Subcategoria,Provincia,Departamento,Localidad,Nombre,...,Mail,Web,InformaciÃ³n adicional,Latitud,Longitud,TipoLatitudLongitud,Fuente,Tipo_gestion,aÃ±o_inicio,AÃ±o_actualizacion
1895,26035030,26,26035,NaN,Bibliotecas Populares,NaN,Chubut,Futaleufu,Esquel,Biblioteca Popular TolkeyÃ©n,...,bibliotecatolkeyen@yahoo.com.ar,NaN,NaN,-42.914574,-71.302231,LocalizaciÃ³n precisa,CONABIP,Comunitaria,2007.0,2018
223,38028040,38,38028,NaN,Bibliotecas Populares,NaN,Jujuy,Humahuaca,Humahuaca,Biblioteca Popular General BartolomÃ© Mitre,...,bib.mitre@yahoo.com.ar,NaN,NaN,-23.205767,-65.348417,LocalizaciÃ³n precisa,CONABIP,Comunitaria,1906.0,2018


CREAR UN ID UNICO UTILIZANDO EL INICE.

In [28]:
df_Biblioteca['Id']=df_Biblioteca.index
df_Biblioteca['Id']=df_Biblioteca['Id'].astype(str)
df_Biblioteca['Id']=df_Biblioteca['Id']+'B'
df_Biblioteca.sample()

,Cod_Loc,IdProvincia,IdDepartamento,Observacion,CategorÃ­a,Subcategoria,Provincia,Departamento,Localidad,Nombre,...,Web,InformaciÃ³n adicional,Latitud,Longitud,TipoLatitudLongitud,Fuente,Tipo_gestion,aÃ±o_inicio,AÃ±o_actualizacion,Id
1621,26014020,26,26014,NaN,Bibliotecas Populares,NaN,Chubut,Cushamen,Cholila,Biblioteca Popular Ruca Raqui Zuam,...,NaN,NaN,-42.511866,-71.430016,LocalizaciÃ³n precisa,CONABIP,Comunitaria,1930.0,2018,1621B


**RENOMBRAR Y ORDENAR COLUMNAS**

In [29]:
df_Biblioteca.rename({'TelÃ©fono':'Telefono','CategorÃ­a':'Categoria','InformaciÃ³n adicional':'Informacion Adicional','aÃ±o_inicio':'Ano_Inicio', 'AÃ±o_actualizacion':'Ano_Actualizacion'},
axis=1, inplace=True)

In [30]:
df_Biblioteca.drop(['Provincia','Departamento','Localidad','IdDepartamento','CP', 'Cod_tel','IdProvincia',],axis=1,inplace=True)

In [31]:
f_Biblioteca=df_Biblioteca[['Id','Cod_Loc',  'Categoria', 
       'Nombre', 'Domicilio',
       'Telefono', 'Mail','Observacion', 'Subcategoria', 'Piso', 'Web', 'Informacion Adicional', 'Latitud', 'Longitud', 'TipoLatitudLongitud',
       'Fuente', 'Tipo_gestion', 'Ano_Inicio', 'Ano_Actualizacion',]]

In [32]:
df_Biblioteca.count()

Cod_Loc                  2017
Observacion                 0
Categoria                2017
Subcategoria                0
Nombre                   2017
Domicilio                2017
Piso                        0
Telefono                 2017
Mail                     2017
Web                         0
Informacion Adicional       0
Latitud                  2017
Longitud                 2017
TipoLatitudLongitud      2017
Fuente                   2017
Tipo_gestion             2017
Ano_Inicio               2010
Ano_Actualizacion        2017
Id                       2017
dtype: int64

**IMPORTACION DE TIPOS DE DATOS QUE UTILIZARE PARA CREAR LA TABLA SIGUIENTE**

In [33]:
from sqlalchemy.types import VARCHAR 
from sqlalchemy.types import INTEGER
from sqlalchemy.types import Text
from sqlalchemy.types import NUMERIC

**CRACION DE TABLA "Biblioteca"**

In [36]:
df_Biblioteca.to_sql('BIBLIOTECA', con=enginok, if_exists='replace', index=False,
          dtype={"Id":VARCHAR, "Cod_Loc": INTEGER, 
          "Categoria":Text,"Observacion":Text, "Subcategoria":Text,"'Informacion Adicional'":Text,  
          "Nombre":Text,  "Telefono":VARCHAR(100), "Web":VARCHAR(100), "Mail": VARCHAR(100),
          "Fuente":VARCHAR(100),'Domicilio':Text,'Piso':Text, "Latitu":NUMERIC, 'Longitud':NUMERIC, 'TipoLatitudLongitud':Text,
          "Tipo_gestion":VARCHAR(100) , "Ano_inicio":INTEGER, "Ano_actualizacion":INTEGER, })

17

In [37]:
text=("""SELECT * FROM "BIBLIOTECA"
    WHERE "Cod_Loc"=86119100""")

muestreo = pd.DataFrame(enginok.execute(text))
muestreo.head()


""


Llave primaria de la tabla Biblioteca

In [38]:
text=("""ALTER TABLE "BIBLIOTECA" ADD PRIMARY KEY ("Id");""")
enginok.execute(text)

**Creando las relaciones correspondientes 'FOREIGN KEY'**

In [40]:
text=("""ALTER TABLE  "BIBLIOTECA"
ADD FOREIGN KEY ( "Cod_Loc")  REFERENCES "Localidad" ( "Cod_Loc") ;""")
enginok.execute(text)